In [11]:
import pandas as pd
from utils import get_song_id, decode_song_id
from pathlib import Path

In [12]:
def move_audio(fn, from_dir, to_dir):
    assert Path(from_dir / fn).exists(), f'{fn} does not exist in {from_dir}'
    Path(to_dir).mkdir(parents=True, exist_ok=True)
    Path(from_dir / fn).rename(to_dir / fn)

In [13]:
# for fn in Path('audio_tmp').glob('*.mp3'):
#     fn = fn.name
#     move_audio(fn, Path('audio_tmp'), Path(f'audio'))

In [14]:
def remove_keyword(keyword, df_fn):
    df = pd.read_csv(df_fn)
    idx_to_remove = []
    for idx, row in df.iterrows():
        yt_title = row['video_title']
        if keyword in yt_title.lower() and keyword not in row['Song'].lower():
            idx_to_remove.append(idx)
    
    df_dropped = df.loc[idx_to_remove]
    df = df.drop(idx_to_remove)
    df.to_csv(df_fn, index=False)

    df_dropped = df_dropped.drop(columns=['query_idx','official','topic','channel_artist_same','keywords','video_title','video_channel','video_url'])
    df_dropped['Failed reason'] = 'No suitable video'
    df_dropped.to_csv(df_fn.replace('.csv', '_dropped.csv'), index=False)

    for idx, row in df_dropped.iterrows():
        year = row['Year']
        title = row['Song']
        artist = row['Artist']
        song_id = get_song_id(year, title, artist)

        song_fn = Path(f'{song_id}.mp3')
        from_dir = Path('audio')
        to_dir = Path('audio_tmp')
        move_audio(song_fn, from_dir, to_dir)

In [15]:
# remove_keyword('practice', 'csv/kpop_chosen.csv')

In [16]:
from pathlib import Path

def get_list_of_availablity(chosen_df_fn):
    chosen_df_fn = Path(chosen_df_fn)
    df = pd.read_csv(chosen_df_fn)
    file_path = Path('audio')
    mp3_list = list(file_path.glob('*.mp3'))
    print(len(mp3_list))

    preserve_idx = []
    no_mp3_idx = []
    valid_mp3 = []
    for idx, row in df.iterrows():
        year = row['Year']
        title = row['Song']
        artist = row['Artist']
        song_id = get_song_id(year, title, artist)
        if not Path(f'audio/{song_id}.mp3').exists():
            no_mp3_idx.append(idx)
        else:
            preserve_idx.append(idx)
            valid_mp3.append(song_id)

    return preserve_idx, no_mp3_idx, valid_mp3

In [17]:
# chosen_df_fn = 'csv/kpop_chosen.csv'
# preserve_idx, no_mp3_idx, _ = get_list_of_availablity(chosen_df_fn)
# df = pd.read_csv(chosen_df_fn)
# df.loc[preserve_idx].to_csv(chosen_df_fn, index=False)
# no_mp3 = df.loc[no_mp3_idx].drop(columns=['query_idx','official','topic','channel_artist_same','keywords','video_title','video_channel','video_url'])
# print(len(no_mp3_idx))
# no_mp3.to_csv('tmp.csv', index=False)

In [18]:
chosen_df_fn = 'csv/kpop_chosen.csv'
preserve_idx, no_mp3_idx, valid_mp3 = get_list_of_availablity(chosen_df_fn)
df = pd.read_csv(chosen_df_fn)

for fn in Path('audio').glob('*.mp3'):
    song_id = fn.stem
    if song_id not in valid_mp3:
        move_audio(fn.name, Path('audio'), Path('audio_tmp'))

5501


In [20]:
len(pd.read_csv('csv/kpop_chosen.csv')), len(list(Path('audio').glob('*.mp3')))

(5477, 5477)